In [16]:
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [17]:
def write_to_submission_file(predicted_labels, out_file, target='target', index_label='session_id'):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns = [target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [23]:
train_df = pd.read_csv(r'data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv(r'data/test_sessions.csv', index_col='session_id')

In [79]:
# Convert the time columns to datetime type from object
times = ['time%s' %i for i in range(1,11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sorting the data by time
train_df.sort_values(by='time1', inplace=True)

train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [37]:
sites = ['site%s' %i for i in range(1,11)]
train_df[sites].fillna(0).astype('int').to_csv('data/train_sessions_text.csv', sep=',', index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('data/test_sessions_text.csv', sep=',', index=None, header=None)

In [42]:
%%time
cv = CountVectorizer(ngram_range=range(1,3), max_features=50000)

with open('data/train_sessions_text.csv') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('data/test_sessions_text.csv') as inp_test_file:
    X_test = cv.transform(inp_test_file)
X_train.shape, X_test.shape

Wall time: 14.6 s


In [43]:
y_train = train_df['target'].astype('int')

In [44]:
time_split = TimeSeriesSplit(n_splits=10)

In [45]:
[(el[0].shape, el[1].shape) for el in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [47]:
logit = LogisticRegression(C=1, random_state=17)

In [49]:
%%time

cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, scoring='roc_auc', n_jobs=-1)

Wall time: 30.7 s


In [52]:
cv_scores, cv_scores.mean()

(array([0.82815753, 0.65005236, 0.87863631, 0.95869806, 0.84196034,
        0.8851763 , 0.93807669, 0.86108835, 0.93543198, 0.91194491]),
 0.8689222828978467)

In [53]:
logit.fit(X_train, y_train)

C:\Users\Shubham\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [54]:
logit_test_pred = logit.predict_proba(X_test)[:, 1]
write_to_submission_file(logit_test_pred, 'data/subm1.csv')

In [55]:
!kaggle competitions submit catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2 -f data/subm1.csv -m "My first time-series submission based on yorko's kernel"

Successfully submitted to Catch Me If You Can



  0%|          | 0.00/2.27M [00:00<?, ?B/s]
  0%|          | 8.00k/2.27M [00:00<01:09, 34.4kB/s]
  5%|4         | 112k/2.27M [00:00<00:46, 48.3kB/s] 
 20%|#9        | 456k/2.27M [00:00<00:27, 68.5kB/s]
 40%|####      | 936k/2.27M [00:00<00:14, 97.2kB/s]
100%|##########| 2.27M/2.27M [00:06<00:00, 387kB/s]


In [85]:
def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [86]:
X_train_new = add_time_features(train_df.fillna(0), X_train)
X_test_new = add_time_features(test_df.fillna(0), X_test)

In [87]:
X_train_new.shape, X_test_new.shape

((253561, 50004), (82797, 50004))

In [88]:
%%time
cv_scores = cross_val_score(logit, X_train_new, y_train, cv=time_split, scoring='roc_auc', n_jobs=-1)

Wall time: 45.7 s


In [89]:
cv_scores, cv_scores.mean()

(array([0.87687983, 0.75707097, 0.92822327, 0.97550693, 0.90328747,
        0.94266138, 0.95924301, 0.93092701, 0.95450394, 0.9439641 ]),
 0.9172267896065526)

In [90]:
logit.fit(X_train_new, y_train)

C:\Users\Shubham\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [92]:
logit_test_pred2 = logit.predict_proba(X_test_new)[:,1]
write_to_submission_file(logit_test_pred2,'data/subm2.csv')

In [93]:
!kaggle competitions submit catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2 -f data/subm2.csv -m "Time series BoW with time1 feature. CV=0.9172"

Successfully submitted to Catch Me If You Can



  0%|          | 0.00/2.28M [00:00<?, ?B/s]
  0%|          | 8.00k/2.28M [00:00<01:07, 35.1kB/s]
  5%|5         | 120k/2.28M [00:00<00:46, 49.3kB/s] 
 20%|#9        | 464k/2.28M [00:00<00:27, 69.7kB/s]
 29%|##9       | 688k/2.28M [00:00<00:17, 98.2kB/s]
100%|##########| 2.28M/2.28M [00:06<00:00, 367kB/s]


In [94]:
# Tuning parameter C

In [95]:
c_values = np.logspace(-2, 2, 10)

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values}, scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [96]:
%%time

logit_grid_searcher.fit(X_train_new, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   46.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.8min finished
C:\Users\Shubham\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 6min 56s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [100]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9182966280619727, {'C': 0.21544346900318834})

In [101]:
logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred3, 'data/subm3.csv')

In [103]:
!kaggle competitions submit catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2 -f data/subm3.csv -m "Time series BoW with time1 feature and optimized hyperparameter C.

Successfully submitted to Catch Me If You Can



  0%|          | 0.00/2.27M [00:00<?, ?B/s]
  0%|          | 8.00k/2.27M [00:00<01:13, 32.1kB/s]
  5%|4         | 112k/2.27M [00:00<00:50, 45.1kB/s] 
 20%|#9        | 456k/2.27M [00:00<00:29, 64.0kB/s]
 29%|##9       | 680k/2.27M [00:00<00:18, 90.1kB/s]
100%|##########| 2.27M/2.27M [00:05<00:00, 435kB/s]
